In [1]:
import pandas as pd
import numpy as np

from scipy.optimize import minimize

np.random.seed(0)

## Davidson model

In [2]:
DELTA = 1

In [3]:
def calculate_prob(home, away, delta, result):
    prod = np.sqrt(home * away)
    sum = home + away
    draw_prob = delta * prod / (sum + prod)
    if result == 0: return (1 - draw_prob) * away / sum
    if result == 1: return draw_prob
    return (1 - draw_prob) * home / sum

In [4]:
def log_likelihood(x, results):
    delta = x[-1]
    log_lik = 0
    for i in results.index:
        result = results.loc[i]
        home, away, result = result
        log_lik -= np.log(calculate_prob(x[home], x[away], delta, result))
    
    return log_lik

In [5]:
forces = pd.DataFrame()
forces['clubs'] = np.arange(20)
forces['forces'] = np.random.normal(loc=1500, scale=100, size=20)

In [6]:
results = pd.DataFrame()
home = np.repeat(np.arange(20), 19)
away = np.array([j for i in range(20) for j in range(20) if i != j])
results['home'] = home
results['away'] = away
results = results \
    .merge(forces, left_on='home', right_on='clubs') \
    .merge(forces, left_on='away', right_on='clubs') \
    .drop(['clubs_x', 'clubs_y'], axis=1) \
    .rename({'forces_x' : 'force_home', 'forces_y' : 'force_away'}, axis=1)

results['draw_prob'] = DELTA * np.sqrt(results['force_home'] * results['force_away'])
results['draw_prob'] /= (results['force_home'] + results['force_away'] + results['draw_prob'])
results['home_prob'] = results['force_home'] / (results['force_home'] + results['force_away'])
results['draw_test'] = np.random.random(size=380)
results['home_test'] = np.random.random(size=380)
results['game_result'] = 1 * (results['draw_prob'] >= results['draw_test']) + \
    2 * (results['draw_prob'] < results['draw_test']) * (results['home_prob'] >= results['home_test'])

results['prob_result'] = 0
for i in results.index:
    home = results.loc[i, 'force_home']
    away = results.loc[i, 'force_away']
    result = results.loc[i, 'game_result']
    results.loc[i, 'prob_result'] += calculate_prob(home, away, DELTA, result)

results

,home,away,force_home,force_away,draw_prob,home_prob,draw_test,home_test,game_result,prob_result
0,0,1,1676.405235,1540.015721,0.333133,0.521202,0.143353,0.326701,1,0.333133
1,2,1,1597.873798,1540.015721,0.333296,0.509219,0.944669,0.232744,2,0.339499
2,3,1,1724.089320,1540.015721,0.332980,0.528197,0.521848,0.614465,0,0.314703
3,4,1,1686.755799,1540.015721,0.333103,0.522738,0.414662,0.033075,2,0.348612
4,5,1,1402.272212,1540.015721,0.333090,0.476592,0.264556,0.015606,1,0.333090
...,...,...,...,...,...,...,...,...,...,...
375,15,0,1533.367433,1676.405235,0.333112,0.477718,0.929291,0.426904,2,0.318584
376,16,0,1649.407907,1676.405235,0.333326,0.495941,0.099615,0.842855,1,0.333326
377,17,0,1479.484174,1676.405235,0.332900,0.468801,0.945302,0.818033,0,0.354363
378,18,0,1531.306770,1676.405235,0.333106,0.477383,0.869489,0.102414,2,0.318364


In [7]:
results = results[['home', 'away', 'game_result']]
x_true = forces['forces'].values
x_true = np.hstack([x_true, DELTA])
log_likelihood(x_true, results)

417.6242818892286

In [8]:
x = np.random.normal(loc=1500, scale=100, size=20)
x = np.hstack([x, 1])
res = minimize(log_likelihood, x0=x, args=(results))
res

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 410.15230531889813
        x: [ 7.819e+02  1.454e+03 ...  7.316e+02  1.084e+00]
      nit: 27
      jac: [-7.782e-04  1.831e-04 ... -6.561e-04  3.929e-04]
 hess_inv: [[ 5.169e-01 -6.628e-03 ...  5.439e-02 -4.961e-04]
            [-6.628e-03  1.006e+00 ...  2.246e-02 -1.405e-03]
            ...
            [ 5.439e-02  2.246e-02 ...  1.842e+00 -2.153e-03]
            [-4.961e-04 -1.405e-03 ... -2.153e-03  4.912e-03]]
     nfev: 1144
     njev: 52

In [9]:
abs(1 - res.x / x_true) * 100

array([53.35749579,  5.57960321, 16.11265816, 21.8487727 ,  1.1062419 ,
        4.32056335,  5.56545561, 16.09038222, 30.02876785, 81.70363796,
       38.79479249, 33.00575155, 12.63265271, 33.19450914, 20.72895631,
       18.41570283, 17.67981552, 34.85542781,  5.07076655, 48.28206428,
        8.43361875])

## Agresti model

In [10]:
DELTA_0, DELTA_1 = -10, -1
DELTAS = [DELTA_0, DELTA_1]

In [11]:
def calculate_prob(home, away, deltas, result):
    delta_0, delta_1 = deltas
    p_leq_0 = np.exp(delta_0 + home) / (np.exp(delta_0 + home) + np.exp(away))
    if result == 0: return p_leq_0
    p_leq_1 = np.exp(delta_1 + home) / (np.exp(delta_1 + home) + np.exp(away))
    if result == 1: return p_leq_1 - p_leq_0
    return 1 - p_leq_1

In [12]:
def log_likelihood(x, results):
    delta_0 = x[-2]
    delta_1 = x[-1]
    deltas = [delta_0, delta_1]
    log_lik = 0
    for i in results.index:
        result = results.loc[i]
        home, away, result = result
        log_lik -= np.log(calculate_prob(x[home], x[away], deltas, result))
    
    return log_lik

In [13]:
forces = pd.DataFrame()
forces['clubs'] = np.arange(20)
forces['forces'] = np.random.normal(loc=150, scale=10, size=20)

In [14]:
results = pd.DataFrame()
home = np.repeat(np.arange(20), 19)
away = np.array([j for i in range(20) for j in range(20) if i != j])
results['home'] = home
results['away'] = away
results = results \
    .merge(forces, left_on='home', right_on='clubs') \
    .merge(forces, left_on='away', right_on='clubs') \
    .drop(['clubs_x', 'clubs_y'], axis=1) \
    .rename({'forces_x' : 'force_home', 'forces_y' : 'force_away'}, axis=1)

results['away_prob'] = np.exp(DELTA_0 + results['force_home']) / (np.exp(DELTA_0 + results['force_home']) + np.exp(results['force_away']))
results['draw_prob'] = np.exp(DELTA_1 + results['force_home']) / (np.exp(DELTA_1 + results['force_home']) + np.exp(results['force_away']))
results['result_test'] = np.random.random(size=380)
results['game_result'] = 1 * (results['result_test'] >= results['away_prob']) + \
    1 * (results['result_test'] >= results['draw_prob'])

results['prob_result'] = 0
for i in results.index:
    home = results.loc[i, 'force_home']
    away = results.loc[i, 'force_away']
    result = results.loc[i, 'game_result']
    results.loc[i, 'prob_result'] += calculate_prob(home, away, DELTAS, result)

results

,home,away,force_home,force_away,away_prob,draw_prob,result_test,game_result,prob_result
0,0,1,152.238436,153.296230,1.576355e-05,1.132672e-01,0.605712,2,0.886733
1,2,1,162.859840,153.296230,3.926016e-01,9.998091e-01,0.115662,0,0.392602
2,3,1,134.930016,153.296230,4.794131e-13,3.884724e-09,0.727888,2,1.000000
3,4,1,156.764607,153.296230,1.454522e-03,9.218950e-01,0.637462,1,0.920440
4,5,1,146.179910,153.296230,3.685335e-08,2.985366e-04,0.811939,2,0.999701
...,...,...,...,...,...,...,...,...,...
375,15,0,148.825255,152.238436,1.495302e-06,1.197152e-02,0.344289,2,0.988028
376,16,0,154.661664,152.238436,5.119501e-04,8.058441e-01,0.548849,1,0.805332
377,17,0,146.297576,152.238436,1.193912e-07,9.665022e-04,0.815225,2,0.999033
378,18,0,145.461960,152.238436,5.176891e-08,4.193119e-04,0.098610,2,0.999581


In [15]:
results = results[['home', 'away', 'game_result']]
x_true = forces['forces'].values
x_true = np.hstack([x_true, DELTAS])
log_likelihood(x_true, results)

70.48100467033719

In [16]:
x = np.random.normal(loc=1500, scale=100, size=20)
x = np.hstack([x, [1, 1]])
res = minimize(log_likelihood, x0=x, args=(results))
res

/var/folders/67/qnnhzk_15ydg7t3gr6_lmx8r0000gq/T/ipykernel_45792/2126339871.py:3: RuntimeWarning: overflow encountered in exp
  p_leq_0 = np.exp(delta_0 + home) / (np.exp(delta_0 + home) + np.exp(away))
/var/folders/67/qnnhzk_15ydg7t3gr6_lmx8r0000gq/T/ipykernel_45792/2126339871.py:3: RuntimeWarning: invalid value encountered in scalar divide
  p_leq_0 = np.exp(delta_0 + home) / (np.exp(delta_0 + home) + np.exp(away))
/var/folders/67/qnnhzk_15ydg7t3gr6_lmx8r0000gq/T/ipykernel_45792/2126339871.py:5: RuntimeWarning: overflow encountered in exp
  p_leq_1 = np.exp(delta_1 + home) / (np.exp(delta_1 + home) + np.exp(away))
/var/folders/67/qnnhzk_15ydg7t3gr6_lmx8r0000gq/T/ipykernel_45792/2126339871.py:5: RuntimeWarning: invalid value encountered in scalar divide
  p_leq_1 = np.exp(delta_1 + home) / (np.exp(delta_1 + home) + np.exp(away))


  message: NaN result encountered.
  success: False
   status: 3
      fun: nan
        x: [ 1.505e+03  1.734e+03 ...  1.000e+00  1.000e+00]
      nit: 0
      jac: [       nan        nan ...        nan        nan]
 hess_inv: [[1 0 ... 0 0]
            [0 1 ... 0 0]
            ...
            [0 0 ... 1 0]
            [0 0 ... 0 1]]
     nfev: 23
     njev: 1

In [17]:
abs(1 - res.x / x_true) * 100

array([ 888.26120702, 1031.11880352,  804.063712  ,  992.44951221,
        880.09885723, 1026.78424184, 1022.9738448 ,  902.971686  ,
        946.73403337,  888.97274294,  867.37770917,  749.36464462,
        836.38201323, 1012.20781144,  892.66472311,  872.17331407,
        893.51373603, 1014.01882095,  964.27241666, 1052.96046937,
        110.        ,  200.        ])